# Apache Spark for astronomy: hands-on session 1

### Context

Welcome to the series of notebooks on Apache Spark! The main goal of this series is to get familiar with Apache Spark, and in particular its Python API called PySpark in a context of the astronomy. In this first notebook, we will introduce few Apache Spark functionalities of interest (and by no means complete!).

### Apache Spark

[Apache Spark](https://spark.apache.org/) is a cluster computing framework, that is a set of tools to perform computation on a network of many machines. Spark started in 2009 as a research project, and it had a huge success so far in the industry. It is based on the so-called MapReduce cluster computing paradigm, popularized by the Hadoop framework using implicit data parallelism and fault tolerance.

The core of Spark is written in Scala which is a general-purpose programming language that has been started in 2004 by Martin Odersky (EPFL). The language is inter-operable with Java and Java-like languages, and Scala executables run on the Java Virtual Machine (JVM). Note that Scala is not a pure functional programming language. It is multi-paradigm, including functional programming, imperative programming, object-oriented programming and concurrent computing.

Spark provides many functionalities exposed through Scala/Python/Java/R API (Scala being the most complete one). As far as this workshop is concerned, we will use the Python API (called PySpark) for obvious reasons. But feel free to put your hands on Scala, it's worth it. For those interested, you can have a look at this [tutorial](https://gitlab.in2p3.fr/MaitresNageurs/QuatreNages/Scala) on Scala.

### Learning objectives

- Loading and distributing data with Spark SQL (Apache Spark Data Sources API)
- Exploring DataFrame & partitioning
- Manipulating Spark SQL built-in functions

## Apache Spark Data Sources

### A tour of data formats

There are many data formats used in the context of Big Data: CSV (1978), XML (1996), JSON (2001), Thrift (2007), Protobuf (2008), Avro & SequenceFile (2009), Parquet (2013), ORC (2016), and the list goes on... Some are _naively_ structured that is using a single type to describe the data (e.g. text) without any internal organisation to access faster the data. Others are more complex and highly optimised for big data treatment (e.g. Parquet). Spark handles most of them by default. Unfortunately those are not the data formats typically chosen by the scientific community. In astronomy for example you would rather store the data in FITS (1981) or HDF5 (1988) format, and in particle physics you would use ROOT (1995). 
These are multi-purposes data formats: images, histograms, spectra, particle lists, data cubes, or even structured data such as multi-table databases can be efficiently stored and accessed.

### Connecting to Data Source

The data source API in Apache Spark belongs to the [Spark SQL module](https://spark.apache.org/sql/). Note that Spark Core has some simple built-in ways to read data from disk (binary or text), but Spark SQL is more complete and give you access to DataFrames directly. If you want to connect a specific data source with Apache Spark, you have mostly two ways:

- [indirect] Access and distribute your files as binary streams (Spark does it natively), and decode the data on-the-fly within executors using third-party libraries.
- [native] Use a built-in or custom connector to access, distribute and decode the data natively.

FITS or HDF5 as most of scientific data formats, were not designed for serialisation (distribution of data over machines) originally and they often use compression to reduce the size on disk. Needless to say that default Spark cannot read those natively. 

First attempts to connect those data formats (see e.g. [1] for FITS) with Spark were using the indirect method above. By reading files as binary streams, the indirect method has the advantage of having access to all FITS functionalities implemented in the underlying user library. This can be an advantage when working with the Python API for example which already contains many great scientific libraries. However this indirect method assumes each Spark mapper will receive and handle one entire file (since the filenames are parallelized and entire file data must be reconstructed from binary once the file has been opened by a Spark mapper). Therefore each single file must fit within the memory of a Spark mapper, hence the indirect method cannot distribute a dataset made of large FITS files (e.g. in [1] they have a 65 GB dataset made of 11,150 files). In addition by assuming each Spark mapper will receive and handle one entire file, the indirect method will have a poor load balancing if the dataset is made of files with not all the same size.

Fortunately Apache Spark low-level layers are sufficiently well written to allow extending the framework and write native connectors for any kind of data sources. Recently connectors for FITS, HDF5 and ROOT were made available [2, 3, 4] to the community. With such connectors, there is a guarantee of having a good load balancing regardless the structure of the dataset and the size of the input files is no more a problem (a 1 TB dataset made of thousand 1 GB files or one single 1 TB file will be viewed as almost the same by a native Spark connector). Note however that the Data Source API is in Java/Scala and if there is no library to play with your data source in those languages you must implement it (what has been done in [2]) or interface with another language.

Note that the low-level layers dealing with the data sources have been recently updated. Apache Spark 2.3 introduced the Data Source API version 2. While the version 1 is still available and usable for a long time, we expect that all Spark connectors will comply with this v2 in the future.

[1] Z. Zhang and K. Barbary and F. A. Nothaft and E. R. Sparks and O. Zahn and M. J. Franklin and D. A. Patterson and S. Perlmutter, Kira: Processing Astronomy Imagery Using Big Data Technology, DOI 10.1109/TBDATA.2016.2599926.  
[2] Peloton, Julien and Arnault, Christian and Plaszczynski, Stéphane, FITS Data Source for Apache Spark, Computing and Software for Big Science (1804.07501). https://github.com/astrolabsoftware/spark-fits   
[3] Liu, Jialin and Racah, Evan and Koziol, Quincey and Canon, Richard Shane, H5spark: bridging the I/O gap between Spark and scientific data formats on HPC systems, Cray user group (2016). https://github.com/valiantljk/h5spark  
[4] Viktor Khristenko, & Jim Pivarski. (2017, October 20). diana-hep/spark-root: Release 0.1.14 (Version v0.1.14). Zenodo. http://doi.org/10.5281/zenodo.1034230

## Spark SQL and DataFrames

### DataFrameReader

The interface to read data from disk is always the same for any kind of built-in and officially supported data format:

```python
df = spark.read\
    .format(format: str)\
    .option(key: str, value: Any)\
    # ...
    .option(key: str, value: Any)\
    .load(path: str)
```
 
Note that for most of the data sources, you can use wrappers such as:

```python
spark.read.csv(path, key1=value1, key2=value2, ...)
```

**Format**: The format can be "csv", "json", "parquet", etc. 

**Options**: The number of options depends on the underlying data source. Each has its own set of options. 
In most of the case, no options are needed, but you might want to explore the different possibilities at some point. Surprisingly it is not easy to find documentation and the best remains to read the source code documentation. In pyspark you can easily access it via the wrappers:

```python
# DataFrameReader object
df_reader = spark.read

# Doc on reading CSV
df_reader.csv?
# doc printed

# Doc on reading Parquet
df_reader.parquet?
# doc printed
```


**Path**: The way to specify path is threefold: either a single file (`path/to/folder/myfile.source`), or an entire folder (`path/to/folder`), or a glob pattern (`path/to/folder/*pattern*.source`). Note that you also need to specify the type of file system you are using. Example:

``` python
# Connect to hdfs
path = 'hdfs:///path/to/data'

# Connect to S3
path = 's3:///path/to/data'

# Connect to local file system
path = 'files:///path/to/data'
```

If nothing is specified (`'/path/to/data'`), it will adapt to your `--master` (e.g. if you launch spark in local mode, you will connect to the local file system by default).


### Using a custom connector

You can also connect to custom connector not included in the default Spark distribution. To do so, you will need to specify the dependencies when submitting your job or invoking your shell. If your connector is available through [Maven Central Repository](https://search.maven.org/), you can easily specify it via:

```
# Direct download from central repository
spark-submit --packages groupId:artifactId:version ...
```

Note that this is the same syntax when launching the `pyspark` shell.
For example, if you want to read FITS files using the [spark-fits](https://github.com/astrolabsoftware/spark-fits) connector you would add the following:

```
# Direct download from central repository
spark-submit --packages com.github.astrolabsoftware:spark-fits_2.11:0.7.1 ...
```

You can find the spark-fits entry in the Maven Central [here](https://search.maven.org/artifact/com.github.astrolabsoftware/spark-fits_2.11/0.7.1/jar) for reference.
Alternatively you can download the source code for a particular connector, compile it and include the `jars`:

```
# Specify manually the dependency
spark-submit --jars /path/to/lib/spark-fits.jars ...
```

Note that when you launch `pyspark`, already a numbers of `jars` are included by default (the ones for Spark for example).

## Loading and distributing data

You will find test data in the folder `data`.

In [1]:
from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

### Loading Data: simply structured data (text)

You can load CSV data into a DataFrame by simply using:

In [4]:
# Load simple CSV file
df_csv = spark.read.format("csv")\
    .load("../../data/clusters.csv")
df_csv.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



Notice by default the CSV connector interprets all entries as String, and give dummy names to columns. You can infer the data type and use the first row as column names by specifying options:

In [5]:
df_csv = spark.read.format("csv")\
    .option('inferSchema', True)\
    .option('header', True)\
    .load("../../data/clusters.csv")
df_csv.printSchema()

root
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)
 |-- id: integer (nullable = true)



In [ ]:
# Make a nice representation of our data
df_csv.show(5)

### Loading Data: complex structured data (Parquet)

More complex data format can infer automatically schema, and data types.
They are also optimised for fast data access and small memory consumption.

In [6]:
# Same using Parquet - Note that the schema and the data types 
# are directly inferred.
df_parquet = spark.read.format("parquet").load("../../data/clusters.parquet")
df_parquet.printSchema()
df_parquet.show(5)

root
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)
 |-- id: integer (nullable = true)

+-------------------+-------------------+------------------+---+
|                  x|                  y|                 z| id|
+-------------------+-------------------+------------------+---+
|-1.4076402686194887|  6.673344773733206| 8.208460943517498|  2|
| 0.6498424376672443|  3.744291410605022|1.0917784706793445|  0|
| 1.3036201950328201|-2.0809475280266656| 4.704460741202294|  1|
|-1.3741641126376476|  4.791424573067701| 2.562770404033503|  0|
| 0.3454761504864363| -2.481008091382492|2.3088066072973583|  1|
+-------------------+-------------------+------------------+---+
only showing top 5 rows



### Loading Data: astronomy format (FITS)

To read FITS, you will need to specify a custom connector such as [spark-fits](https://github.com/astrolabsoftware/spark-fits) (this is done for you):

```
PYSPARK_DRIVER_PYTHON=jupyter-notebook pyspark --packages com.github.astrolabsoftware:spark-fits_2.11:0.8.3 ...
```

In [7]:
df_fits = spark.read.format("fits").option("hdu", 1).load("../../data/clusters.fits")
df_fits.printSchema()
df_fits.show(5)

root
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)
 |-- id: integer (nullable = true)

+-------------------+-------------------+------------------+---+
|                  x|                  y|                 z| id|
+-------------------+-------------------+------------------+---+
|-1.4076402686194887|  6.673344773733206| 8.208460943517498|  2|
| 0.6498424376672443|  3.744291410605022|1.0917784706793445|  0|
| 1.3036201950328201|-2.0809475280266656| 4.704460741202294|  1|
|-1.3741641126376476|  4.791424573067701| 2.562770404033503|  0|
| 0.3454761504864363| -2.481008091382492|2.3088066072973583|  1|
+-------------------+-------------------+------------------+---+
only showing top 5 rows



## Partitioning

You might noticed Spark cut out the dataset into partitions, and for each partition Spark will run one task.
Following the principle that moving computation is usually cheaper than moving data, Spark reads file blocks in a performant way: instead of copying file blocks to a central compute node, which can be expensive, the driver sends the computation to worker nodes close to DataNodes where the data reside.
Normally, Spark tries to set the number of partitions automatically based on your distributed file system configuration. For example in HDFS, the size of data blocks is typically 128 MB (tunable), therefore the default number of Spark partitions when reading data will be the total number of 128 MB chunks for your dataset.

```
How many partitions should I use?
```

There is no unique answer to that. You will often hear: `typically you want 2-4 partitions for each CPU in your cluster`, but that implies you can accomodate infinite number of CPUs at limited partition size. In practice it will mainly depend on: 
- the total volume of data you want to distribute, 
- the number of CPU you have access to and their RAM, 
- and the kind of task you want to perform.

If you have too few partitions, you will not take benefit from all of the cores available in the cluster (time to solution can be longer, and you can run out of memory for intensive tasks). 
If you have too many partitions, there will be excessive overhead in managing many small tasks.
In between, you are generally good.
Note that when you load data, Spark assign itself the number of partitions, and you can repartition the dataset using:

```python
# numPartitions is arbitrary but
# this operation will add a shuffle step
df.repartition(numPartitions)

# Using either a number of partition or 
# column names to repartition by range
df.repartitionByRange(numPartitions, colnames)

# Using one or several columns to repartition
df.orderBy(colnames)

# numPartitions must be lower than the 
# current one, but no shuffle is performed
df.coalesce(numPartitions)
```

You can access the number of partitions in use using:

```python
df.rdd.getNumPartitions()
```

Frequent basic use-cases:
- The standard: You have a lot of data stored in large files and data entries need to be process independently from each other --> keep the default.
- The multi-files: When reading many small files (each being much smaller than the typical 128 MB data block size), you usually end up with way more partitions than if you were reading the same volume of data but with fewer files --> repartition your dataset with fewer partitions.
- The shuffle: If your tasks involve a lot of data movement and communication between machines (data shuffle) --> it is usually a good idea to keep the number of partitions not too high.
- The heavy filter: sometimes you filter out a lot of data based on some condition, and then you execute some action on the remaining subset. Because of the filering, you might end up with many empty partitions --> try to see if repartitioning with fewer partitions helps in processing the remaining faster.

**In practice you will end up experimenting a bit with the number of partitions... But always keep in mind the main reason to repartition is to minimize data movement inside the cluster.**

## Basic operations on DataFrames

Let's load our data

In [8]:
df = spark.read.format("parquet").load("../../data/clusters.parquet")

### Select & filters

There are powerful methods to select subsets of columns or to filter rows based on values. Note that column selection and row filtering are transformations (in the sense of functional programming) - nothing really happens to the data until you trigger an action.

In [9]:
# Filtering rows based on entry values
df_x_more_than_one = df.filter("x > 1")

# Same as before, but different syntax
df_x_more_than_one = df.filter(df["x"] > 1)

# Filtering column based on their name
df_y_only = df.select('y')
df_x_and_y = df.select(['x', 'y'])

# You can chain transformations
df_x_cluster_one = df.select('x').filter('id == 1')

# Trigger an action
row_with_x_more_than_one = df_x_more_than_one.count()
print("{} entries with x > 1".format(row_with_x_more_than_one))

1291 entries with x > 1


### Map and mapPartitions

You can also apply transformation on DataFrame values. The most simple transformation would use the `map` method which preserves the cardinality of the DataFrame. `mapPartitions` is similar, although the cardinality is not preserved.

In [10]:
# Example for map: multiply all elements by 2
def multiply_by_two(row):
    """
    """
    new_row = [2*i for i in row]
    return new_row

# map is a RDD method (not available for DataFrame in pyspark)
df.rdd.map(multiply_by_two).toDF(df.columns).show(5)

+-------------------+------------------+------------------+---+
|                  x|                 y|                 z| id|
+-------------------+------------------+------------------+---+
|-2.8152805372389773|13.346689547466411|16.416921887034995|  4|
| 1.2996848753344885| 7.488582821210044| 2.183556941358689|  0|
| 2.6072403900656402|-4.161895056053331| 9.408921482404589|  2|
| -2.748328225275295| 9.582849146135402| 5.125540808067006|  0|
| 0.6909523009728726|-4.962016182764984| 4.617613214594717|  2|
+-------------------+------------------+------------------+---+
only showing top 5 rows



In [11]:
# Example for mapPartitions: count the number of rows per partition
def yield_num_rows(part, param1):
    """ Yield the number of rows in the partition
    
    Parameters
    ----------
    part : Iterator
        Iterator containing partition data
        
    Yield
    ----------
    length: integer
        number of rows inside the partition
    """
    partition_data = [*part]
    print(param1)
    yield len(partition_data)

# Let's repartition our DataFrame in 12 partitions
df_repart = df.repartition(12)

# mapPartitions is a RDD method(not available for DataFrame in pyspark)
print("Number of rows per Spark partitions:")
# df_repart.rdd.mapPartitions(lambda part: yield_num_rows(part)).collect()
param1 = 2
df_repart.rdd.mapPartitions(lambda part: yield_num_rows(part, param1)).collect()

Number of rows per Spark partitions:


[333, 334, 334, 334, 334, 333, 333, 333, 333, 333, 333, 333]

notice the super good load balancing! Be careful though in using `collect`, as data flows from the executors to the (poor and lonely and undersized) driver. Always reducing the data first!

**Exercise (££)**: Compute the barycentre of each partition (hint: repartition or re-order according to the `id` column).

In [12]:
import numpy as np

def yield_barycentre(part):
    """ Yield the number of rows in the partition
    
    Parameters
    ----------
    part : Iterator
        Iterator containing partition data
        
    Yield
    ----------
    length: integer
        number of rows inside the partition
    """
    try:
        partition_data = [*part]
        x, y, z, _ = np.transpose(partition_data)
        yield np.mean([x, y, z], axis=1)
    except ValueError as e:
        # Empty partition
        yield [None, None, None]

# Let's repartition our DataFrame according to "id"
df_repart = df.orderBy("id")

# mapPartitions is a RDD method(not available for DataFrame in pyspark)
print("Cluster coordinates:")
df_repart.rdd.mapPartitions(yield_barycentre).collect()

Cluster coordinates:


[array([1.00131431, 4.25087991, 2.02169007]),
 array([ 0.90843113, -1.53356089,  2.92620126]),
 array([-1.23649382,  7.78371632,  9.29293767]),
 [None, None, None]]

### Statistics

You can easily access basics statistics of your DataFrame:

In [13]:
# The describe method returns a DataFrame
df.describe().show()

+-------+-------------------+------------------+-------------------+------------------+
|summary|                  x|                 y|                  z|                id|
+-------+-------------------+------------------+-------------------+------------------+
|  count|               4000|              4000|               4000|              4000|
|   mean|0.22461143161189448|3.5005327477749812|  4.746261685611487|           0.99975|
| stddev| 1.4333802737826418| 3.970358011802803| 3.3858958227831852|0.8166496596868619|
|    min| -4.320974828599122|-5.207575440768161|-1.4595005976690572|                 0|
|    max|  4.077800662643146|10.854512466048538| 12.602016902866591|                 2|
+-------+-------------------+------------------+-------------------+------------------+



### Aggregation

Apache Spark has built-in method to perform aggregation. Be careful though - this implies shuffle (i.e. communication between machines and data transfer), and can be a performance killer!

**Exercise (£):** group by `id`, and count the number of elements per `id`

In [14]:
df.groupBy("id").count().show()

+---+-----+
| id|count|
+---+-----+
|  1| 1333|
|  2| 1333|
|  0| 1334|
+---+-----+



### Direct acyclic graph (DAG)

As quickly highlighted above, Spark commands are either transformations (filter, select, ...) or actions (show, take, ...). You can chain actions, and in the end you trigger the computation with an action. Before running any action, Spark will build a graph of the commands, called Direct Acyclic Graph, and... it will do some magic for you. 

**Exercise (£):** Look at the two commands and output. Do you notice the magic?

In [15]:
df.groupBy("id").count().filter('id >= 1').explain()

== Physical Plan ==
*(2) HashAggregate(keys=[id#81], functions=[count(1)])
+- Exchange hashpartitioning(id#81, 200)
   +- *(1) HashAggregate(keys=[id#81], functions=[partial_count(1)])
      +- *(1) Project [id#81]
         +- *(1) Filter (isnotnull(id#81) && (id#81 >= 1))
            +- *(1) FileScan parquet [id#81] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/home/jovyan/work/data/clusters.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(id), GreaterThanOrEqual(id,1)], ReadSchema: struct<id:int>


In [16]:
df.filter('id >= 1').groupBy("id").count().explain()

== Physical Plan ==
*(2) HashAggregate(keys=[id#81], functions=[count(1)])
+- Exchange hashpartitioning(id#81, 200)
   +- *(1) HashAggregate(keys=[id#81], functions=[partial_count(1)])
      +- *(1) Project [id#81]
         +- *(1) Filter (isnotnull(id#81) && (id#81 >= 1))
            +- *(1) FileScan parquet [id#81] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/home/jovyan/work/data/clusters.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(id), GreaterThanOrEqual(id,1)], ReadSchema: struct<id:int>
